## this is the recommendation based on constraint

In [50]:
import pandas as pd

In [51]:
df = pd.read_csv("../data/datasets/updated/tourism_with_id_updated.csv")

In [52]:
df.head()

,Place_Id,Place_Name,Description,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,new_category
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,"['Budaya', 'Keluarga']"
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,['Budaya']
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,"['Hiburan', 'Keluarga']"
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,"['Budaya', 'Hiburan']"
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,"['Petualangan', 'Hiburan']"


In [53]:
unused_column = ["Description", "Time_Minutes", "Coordinate"]

In [54]:
df.drop(unused_column, axis=1,inplace=True)

In [55]:
# knowledge based on paper
def mood_constraint(mood):
    mood = mood.capitalize()
    preferences = []
    if mood == "Senang" or mood == "Sedih":
        preferences = ["Petualangan", "Keluarga", "Romantis", "Budaya"]
    elif mood == "Tenang" or mood == "Marah":
        preferences = ["Alam", "Hiburan", "Olahraga", "Relaksasi"]
    
    return preferences

In [56]:
def filter_by_budget(df, column_name, category):
    category = category.capitalize()
    quartiles = df[column_name].quantile([0.25, 0.5, 0.75])
    q1 = quartiles[0.25]
    q2 = quartiles[0.5]
    q3 = quartiles[0.75]

    if category == "Low":
        filtered_df = df[df[column_name] <= q1]
    elif category == "Medium":
        filtered_df = df[(df[column_name] > q1) & (df[column_name] <= q2)]
    elif category == "High":
        filtered_df = df[((df[column_name] > q2) & (df[column_name] <= q3)) | (df[column_name] > q3) ]
    elif category == "Random":
        filtered_df = df
    else:
        filtered_df = []

    return filtered_df

In [57]:
def filter_by_city(df, city):
    city = city.capitalize()
    filtered_df = df[df["City"] == city]
    return filtered_df

In [58]:
def knowledge_recommender(df, mood, budget, city):
    recommended_destinations = []
    
    mood_preferences = mood_constraint(mood)
    
    # filter dataset based on budget
    filtered_df = filter_by_budget(df,"Price",budget)

    # filter dataset based on city
    filtered_df = filter_by_city(filtered_df,city)

    #print(filtered_df)
    recommended_destinations = filtered_df[filtered_df["new_category"].apply(lambda x: any(preference in x for preference in mood_preferences))]

    
    return recommended_destinations

In [59]:
knowledge_recommendation = knowledge_recommender(df, 'Tenang', 'High', "Bandung")

In [60]:
knowledge_recommendation.sort_values("Rating", ascending=False, inplace=True)

In [61]:
dropped_column = ["Place_Name", "City", "Price", "Lat", "Long", "new_category","Rating"]

In [62]:
knowledge_recommendation.drop(dropped_column, axis=1, inplace=True)

In [63]:
knowledge_recommendation.head(3)

,Place_Id
261,262
288,289
269,270


In [64]:
def give_scoring(df):
    # this function will return score for each user recommended items
    df['score'] = 1
    return df

In [65]:
knowledge_recommendation = give_scoring(knowledge_recommendation)

In [66]:
knowledge_recommendation.to_csv('../data/output/knowledge_recommendation.csv', index=False)